In [1]:
import pandas as pd

import findlay2025a as f25a
import wisc_ecephys_tools as wet

In [2]:
s3 = wet.get_sglx_project("shared")
nbsh = wet.get_sglx_project("seahorse")
experiment = wet.rats.constants.SleepDeprivationExperiments.NOD
sortings = f25a.units.get_nod_sortings()

In [ ]:
for subject, probes in sortings:
    print(f"\n Doing {subject}...")
    savefile = nbsh.get_experiment_subject_file(
        experiment, subject, "firing_rates_by_condition.pqt"
    )
    hgs = f25a.hypnograms.load_statistical_condition_hypnograms(
        experiment, subject, include_full_conservative=True
    )
    mps = f25a.units.load_nod_multiprobe_sorting(
        subject, **f25a.units.get_threshold_kwargs()["mua"]
    )
    trains = mps.get_cluster_trains()
    df = (
        pd.DataFrame(
            {
                condition: {
                    id: hg.covers_time(train).sum() / hg["duration"].sum()
                    for id, train in trains.items()
                }
                for condition, hg in hgs.items()
            }
        )
        .rename_axis("cluster_id")
        .reset_index()
    )
    df.to_parquet(savefile)
    del trains


 Doing CNPIX10-Charles...
quality: {'mua', 'good', nan} excludes 98 clusters.
firing_rate: (0.5, inf) excludes 149 clusters.
182/553 clusters excluded by jointly applying filters. 371 remain.
quality: {'mua', 'good', nan} excludes 138 clusters.
firing_rate: (0.5, inf) excludes 85 clusters.
161/291 clusters excluded by jointly applying filters. 130 remain.


Loading trains by cluster_id: 100%|██████████| 501/501 [03:54<00:00,  2.14it/s]


In [ ]:
subj_dfs = []
for subject, probes in sortings:
    savefile = nbsh.get_experiment_subject_file(
        experiment, subject, "firing_rates_by_condition.pqt"
    )
    subj_df = pd.read_parquet(savefile)
    subj_df["subject"] = subject
    subj_df["experiment"] = experiment
    subj_dfs.append(subj_df)
frs = pd.concat(subj_dfs, axis=0, ignore_index=True)
frs.to_parquet(nbsh.get_project_file("firing_rates_by_condition.pqt"))

In [4]:
frs = pd.read_parquet(nbsh.get_project_file("firing_rates_by_condition.pqt"))

In [10]:
cell_types = pd.read_parquet(nbsh.get_project_file("cell_types.pqt"))
cluster_quality = pd.read_parquet(nbsh.get_project_file("cluster_quality.pqt"))
mps_metrics = pd.read_parquet(nbsh.get_project_file("mps_metrics.pqt"))

id_cols = ["subject", "experiment", "probe", "cluster_id"]
df = pd.merge(cell_types, cluster_quality, on=id_cols, how="left")
df = pd.merge(df, mps_metrics, on=id_cols, how="left")
df = pd.merge(df, frs, on=["subject", "cluster_id"], how="right")

In [11]:
state_fr_cols = [c for c in mps_metrics.columns if "firing_rate_" in c]
df = df.drop(columns=["firing_rate"], errors="ignore")

cond_fr_cols = [c for c in frs.columns if c not in id_cols]
df = pd.melt(
    df,
    id_vars=[col for col in df.columns if col not in cond_fr_cols + state_fr_cols],
    value_vars=cond_fr_cols + state_fr_cols,
    var_name="condition",
    value_name="firing_rate",
)
df["condition"] = df["condition"].str.replace("firing_rate_", "", regex=False)


In [ ]:
strictness = "sua_moderate"
strictness_levels = ["mua", "sua_permissive", "sua_moderate", "sua_conservative"]
strictness_index = strictness_levels.index(strictness)
df = df.loc[df["max_quality"].isin(strictness_levels[strictness_index:])]

In [ ]:
df.to_parquet(nbsh.get_project_file(f"frs.with_meta.{strictness}.pqt"))